In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
train_df = pd.read_csv(Path('Resources/Generator/2019loans.csv'))
test_df = pd.read_csv(Path('Resources/Generator/2020Q1loans.csv'))

In [3]:
# Convert categorical data to numeric and separate target feature for training data
ry = pd.get_dummies(train_df['target'])['high_risk']
rtrimmed = train_df.drop('target', axis=1)
rX = pd.get_dummies(rtrimmed)

# Convert categorical data to numeric and separate target feature for testing data
sy = pd.get_dummies(test_df['target'])['high_risk']
strimmed = test_df.drop('target', axis=1)
sX = pd.get_dummies(strimmed)


In [4]:
# test for missing dummy variables in sets
missing = []
for label in rX.columns:
    if label not in sX.columns:
        missing.append(label)
print(missing)
# confirm only rdum has missing values
print(set(rX.columns) ^ set(sX.columns))

# add missing dummy variables to testing set
for column in missing:
    sX[column] = 0

# confirm values are now symmetric
print(set(rX.columns) ^ set(sX.columns))

['debt_settlement_flag_Y']
{'debt_settlement_flag_Y'}
set()


In [14]:
# Train the Logistic Regression model on the unscaled data and print the model score
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(max_iter=20000)
classifier.fit(rX, ry)
print(f"Training Data Score: {classifier.score(rX, ry)}")
print(f"Testing Data Score: {classifier.score(sX, sy)}")

Training Data Score: 0.7087848932676519
Testing Data Score: 0.5652913653764355


In [6]:
# Train a Random Forest Classifier model and print the model score


In [7]:
# Scale the data

In [8]:
# Train the Logistic Regression model on the scaled data and print the model score

In [9]:
# Train a Random Forest Classifier model on the scaled data and print the model score